In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json    # for handling json files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print("-----#---------#-----------")
print("Libraries Imported")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

-----#---------#-----------
Libraries Imported


In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [3]:
# Load the data
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
type(newyork_data)

dict

In [5]:
neighbourhood_data = newyork_data['features']
neighbourhood_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [6]:
# Transform the data from python dictionaries to pandas dataframe

column_names = ['borough', 'neighbourhood', 'latitude', 'longitude']

neighbourhoods = pd.DataFrame(columns=column_names)
neighbourhoods

,borough,neighbourhood,latitude,longitude


In [7]:
# Now loop through neighbourhood_data to add training examples to dataframe
for entry in neighbourhood_data:
    borough = entry['properties']['borough']
    neighbourhood = entry['properties']['name']
    coordinates = entry['geometry']['coordinates']
    latitude = coordinates[1]
    longitude = coordinates[0]
    
    # Now append these values to the dataframe
    neighbourhoods = neighbourhoods.append({'borough': borough, 'neighbourhood': neighbourhood, 
                                            'latitude':latitude, 'longitude':longitude}, ignore_index=True)

In [8]:
print("Dimensions of neighbourhoods: {}".format(neighbourhoods.shape))
print("Number of unique borough: {}".format(len(neighbourhoods['borough'].unique())))
neighbourhoods.head()

Dimensions of neighbourhoods: (306, 4)
Number of unique borough: 5


,borough,neighbourhood,latitude,longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
# use geopy library to get latitude and longitude values of newyork city

address = 'New York City, NY'

geolocator = Nominatim(user_agent='ny_explorer')
location = geolocator.geocode(address)
latitute = location.latitude
longitude = location.longitude
print("Geographical coordinated of NYC: Latitude: {} and Longitude: {}".format(latitude, longitude))

Geographical coordinated of NYC: Latitude: 40.61731079252983 and Longitude: -74.0060152


In [10]:
# Create a map of NY with neighbourhoods superimposed on it

map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, long, borough, neighbourhood in zip(neighbourhoods['latitude'], 
                                             neighbourhoods['longitude'], 
                                             neighbourhoods['borough'], 
                                             neighbourhoods['neighbourhood']):
    labels = '{}, {}'.format(borough, neighbourhood)
    labels = folium.Popup(labels, parse_html=True)
    
    folium.CircleMarker(location=[lat,long], radius=5, 
                        popup=labels, color='blue', fill=True, 
                        fill_color='#3186cc',
                       fill_opacity = 0.7,
                       parse_html=False).add_to(map_newyork)

map_newyork

In [11]:
# for simplicity we'll focus on Manhattan region and cluster neighbouhoods in that region
manhattan = neighbourhoods[neighbourhoods['borough'] == 'Manhattan'].reset_index(drop=True)
manhattan.head()

,borough,neighbourhood,latitude,longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [12]:
# Now lets get the geographical coordinates of Manhattan
address_2 = 'Manhattan, NY'

geoloctor = Nominatim(user_agent='ny_explorer')
location = geolocator.geocode(address_2)
latitude = location.latitude
longitude = location.longitude
print("Latitude: {} and Longitude: {}".format(latitude, longitude))

Latitude: 40.7896239 and Longitude: -73.9598939


In [13]:
# Lets visualize the neighbourhoods from Manhattan first

map_manhattan = folium.Map(location=(latitude, longitude), zoom_start=11)

for lat, long, borough, neighbourhood in zip(manhattan['latitude'],
                                            manhattan['longitude'],
                                            manhattan['borough'],
                                            manhattan['neighbourhood']):
    label = '{}, {}'.format(borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(location=[lat,long], 
                        radius=5, 
                        popup=label, 
                        color='blue',
                        fill=True, 
                        fill_color='#3186cc', 
                        fill_opacity=0.7, 
                        parse_html=False).add_to(map_manhattan)

map_manhattan

In [14]:
# Definign Foursquare credentials
CLIENT_ID = 'IUPERZI31AI3JRB4RWHAT23TIAS4HGMXTE4X0CPNJSVKIIJT' # your Foursquare ID
CLIENT_SECRET = 'UMKX2IMOX1YTPBFW50FGMPSVTHPEYKKGH5SKXM5NXH0KOGZD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IUPERZI31AI3JRB4RWHAT23TIAS4HGMXTE4X0CPNJSVKIIJT
CLIENT_SECRET:UMKX2IMOX1YTPBFW50FGMPSVTHPEYKKGH5SKXM5NXH0KOGZD


In [15]:
# Exploring the neighbourhoods present in Manhattan
# write a function to get nearby venues within 500 meters for all neighbourhoods

def get_nearby_venues(names, latitude, longitude, radius=500):
    
    # we wants venue list
    venue_list = []
    
    for name, lat, lng in zip(names, latitude, longitude):
        # Creating request url
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venue_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        nearby_venues = pd.DataFrame([item for venue_list in venue_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
manhattan_venues = get_nearby_venues(manhattan['neighbourhood'], latitude=manhattan['latitude'],
                                    longitude=manhattan['longitude'])
manhattan_venues.shape

(3272, 7)

In [17]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym


In [18]:
manhattan_venues.groupby(by='Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,91,91,91,91,91,91
Carnegie Hill,95,95,95,95,95,95
Central Harlem,46,46,46,46,46,46
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,40,40,40,40,40,40
East Village,100,100,100,100,100,100


In [19]:
# how many unique venue categoies we get
print("Number of unique venue categories: {}".format(len(manhattan_venues['Venue Category'].unique())))

Number of unique venue categories: 333


In [20]:
# To cluster the neighbourhoods, 
# we need to form a dataframe of neighbour name and different venue categories present in them

df = pd.get_dummies(data=manhattan_venues[['Venue Category']], prefix='', prefix_sep='')

df['Neighborhood'] = manhattan_venues['Neighborhood']

fixed_columns = [df.columns[-1]] + list(df.columns[:-1])

df = df[fixed_columns]

df.head()

,Neighborhood,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping S

In [21]:
df.shape

(3272, 334)

In [22]:
# Group the neighbourhood by taking the mean of frequency occurence 
df_grouped = df.groupby(by='Neighborhood').mean().reset_index()
print("Dimensions of df_grouped: {}".format(df_grouped.shape))
df_grouped.head()

Dimensions of df_grouped: (40, 334)


,Neighborhood,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping S

In [23]:
# Lets print each neighbourhood along with top 5 most commom venues

count = 5

for hood in df_grouped['Neighborhood']:
    print("-----------"+hood+"-------------")
    temp = df_grouped[df_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype('float')
    print(temp.sort_values(by='freq', ascending=False).reset_index(drop=True).head())

-----------Battery Park City-------------
            venue      freq
0            Park  0.087912
1  Clothing Store  0.065934
2     Coffee Shop  0.054945
3           Hotel  0.054945
4             Gym  0.032967
-----------Carnegie Hill-------------
         venue      freq
0  Coffee Shop  0.073684
1         Café  0.052632
2    Wine Shop  0.042105
3  Pizza Place  0.042105
4  Yoga Studio  0.031579
-----------Central Harlem-------------
                  venue      freq
0    African Restaurant  0.065217
1    Chinese Restaurant  0.043478
2   American Restaurant  0.043478
3                   Bar  0.043478
4  Gym / Fitness Center  0.043478
-----------Chelsea-------------
                 venue  freq
0               Bakery  0.05
1          Coffee Shop  0.05
2  American Restaurant  0.04
3          Art Gallery  0.04
4                Hotel  0.03
-----------Chinatown-------------
                 venue  freq
0   Chinese Restaurant  0.09
1               Bakery  0.07
2         Cocktail Bar  0.05
3  

In [24]:
# sort the venues in descending order
def return_most_common_venues(row, limit):
    row = row.iloc[1:]
    row_sorted = row.sort_values(ascending=False)
    return row_sorted.index.values[:limit]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_grouped['Neighborhood']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Clothing Store,Coffee Shop,Hotel,Gym,Women's Store,Playground,Memorial Site,Gourmet Shop,Plaza
1,Carnegie Hill,Coffee Shop,Café,Wine Shop,Pizza Place,Yoga Studio,Gym / Fitness Center,Bookstore,Cosmetics Shop,Gym,Bar
2,Central Harlem,African Restaurant,Chinese Restaurant,American Restaurant,Bar,Gym / Fitness Center,Seafood Restaurant,French Restaurant,Fried Chicken Joint,Dessert Shop,Music Venue
3,Chelsea,Bakery,Coffee Shop,American Restaurant,Art Gallery,Hotel,Seafood Restaurant,French Restaurant,Ice Cream Shop,Wine Shop,Cocktail Bar
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Dessert Shop,Hotpot Restaurant,Salon / Barbershop,Optical Shop,Noodle House,Dim Sum Restaurant


## ``Cluster Neighbourhoods``

In [26]:
num_clusters = 5

manhattan_clustering = df_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(manhattan_clustering)

# check cluster labels generated for each row in dataframe
kmeans.labels_[:10]

array([1, 2, 0, 3, 0, 1, 1, 2, 0, 1], dtype=int32)

In [27]:
manhattan.head()

,borough,neighbourhood,latitude,longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [28]:
# merge the dataframes to plot with labels
neighborhoods_venues_sorted.insert(loc=0, column='Cluster_labels', value=kmeans.labels_)

manhattan_merged = manhattan

manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighbourhood')

manhattan_merged.head()

,borough,neighbourhood,latitude,longitude,Cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Gym,Sandwich Place,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant,Coffee Shop,Department Store
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Dessert Shop,Hotpot Restaurant,Salon / Barbershop,Optical Shop,Noodle House,Dim Sum Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Café,Bakery,Grocery Store,Bank,Mobile Phone Shop,Tapas Restaurant,Pizza Place,Donut Shop,Supplement Shop,Park
3,Manhattan,Inwood,40.867684,-73.921210,2,Mexican Restaurant,Café,Restaurant,Chinese Restaurant,Pizza Place,Caribbean Restaurant,Bakery,Wine Bar,Park,Deli / Bodega
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Latin American Restaurant,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar,Bakery


In [29]:
# Finally Finally lets visualize our clusters

manhattan_map_clustered = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# on the map
marker_colors = []
for lat, lng, borough, neighbourhood, cluster in zip(manhattan_merged['latitude'],
                                           manhattan_merged['longitude'],
                                           manhattan_merged['borough'],
                                           manhattan_merged['neighbourhood'],
                                                     manhattan_merged['Cluster_labels']):
    label = '{}, {}, {}'.format(neighbourhood, borough, cluster)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(location=[lat,lng],
                       radius=5,
                       popup=label,
                       color = rainbow[cluster-1],
                       fill=True,
                        fill_color=rainbow[cluster-1],
                        fill_opacity=0.7).add_to(manhattan_map_clustered)
    
    


manhattan_map_clustered

#### ``Examining Clusters``

In [30]:
# Cluster 1
manhattan_merged.loc[manhattan_merged['Cluster_labels'] == 0, 
                     manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Dessert Shop,Hotpot Restaurant,Salon / Barbershop,Optical Shop,Noodle House,Dim Sum Restaurant
6,Central Harlem,African Restaurant,Chinese Restaurant,American Restaurant,Bar,Gym / Fitness Center,Seafood Restaurant,French Restaurant,Fried Chicken Joint,Dessert Shop,Music Venue
9,Yorkville,Italian Restaurant,Coffee Shop,Bar,Gym,Sushi Restaurant,Deli / Bodega,Diner,Japanese Restaurant,Ice Cream Shop,Wine Shop
12,Upper West Side,Italian Restaurant,Bar,Bakery,Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Bagel Shop,Café,Wine Bar
19,East Village,Bar,Pizza Place,Speakeasy,Mexican Restaurant,Ice Cream Shop,Korean Restaurant,Italian Restaurant,Wine Bar,Cocktail Bar,Coffee Shop
25,Manhattan Valley,Mexican Restaurant,Coffee Shop,Bar,Vietnamese Restaurant,Pizza Place,French Restaurant,Japanese Restaurant,Hawaiian Restaurant,Grocery Store,Furniture / Home Store
27,Gramercy,Bar,Bagel Shop,Italian Restaurant,Coffee Shop,Pizza Place,Ice Cream Shop,Thrift / Vintage Store,Playground,Cocktail Bar,Diner


In [31]:
# cluster 2
manhattan_merged.loc[manhattan_merged['Cluster_labels'] == 1, 
                     manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Lincoln Square,Plaza,Café,Concert Hall,Theater,Gym / Fitness Center,Performing Arts Venue,Coffee Shop,Bakery,Food Truck,Park
14,Clinton,Theater,Italian Restaurant,American Restaurant,Gym / Fitness Center,Coffee Shop,Sandwich Place,Wine Shop,Cocktail Bar,Spa,Hotel
15,Midtown,Hotel,Coffee Shop,Steakhouse,Sporting Goods Shop,Clothing Store,Theater,Bookstore,Sandwich Place,Café,Cuban Restaurant
16,Murray Hill,Hotel,Sandwich Place,Japanese Restaurant,American Restaurant,Coffee Shop,Taco Place,Sushi Restaurant,Gym / Fitness Center,Bar,Pizza Place
21,Tribeca,American Restaurant,Park,Spa,Wine Bar,Italian Restaurant,Café,Greek Restaurant,Scenic Lookout,Hotel,Basketball Court
28,Battery Park City,Park,Clothing Store,Coffee Shop,Hotel,Gym,Women's Store,Playground,Memorial Site,Gourmet Shop,Plaza
29,Financial District,Coffee Shop,Pizza Place,Cocktail Bar,American Restaurant,Gym,Steakhouse,Hotel,Italian Restaurant,Bar,Park
32,Civic Center,Coffee Shop,Spa,Gym / Fitness Center,American Restaurant,Cocktail Bar,French Restaurant,Bakery,Park,Italian Restaurant,Hotel
33,Midtown South,Korean Restaurant,Hotel,Hotel Bar,Cosmetics Shop,Coffee Shop,Dessert Shop,Gym / Fitness Center,Japanese Restaurant,American Restaurant,Cocktail Bar
35,Turtle Bay,Coffee Shop,Italian Restaurant,Café,Park,Steakhouse,Sushi Restaurant,Seafood Restaurant,Deli / Bodega,Hotel,Ramen Restaurant


In [32]:
# cluster 3
manhattan_merged.loc[manhattan_merged['Cluster_labels'] == 2, 
                     manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Gym,Sandwich Place,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant,Coffee Shop,Department Store
2,Washington Heights,Café,Bakery,Grocery Store,Bank,Mobile Phone Shop,Tapas Restaurant,Pizza Place,Donut Shop,Supplement Shop,Park
3,Inwood,Mexican Restaurant,Café,Restaurant,Chinese Restaurant,Pizza Place,Caribbean Restaurant,Bakery,Wine Bar,Park,Deli / Bodega
4,Hamilton Heights,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Latin American Restaurant,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar,Bakery
5,Manhattanville,Deli / Bodega,Coffee Shop,Bar,Mexican Restaurant,Italian Restaurant,Burger Joint,Thrift / Vintage Store,Food & Drink Shop,Cuban Restaurant,Boutique
7,East Harlem,Mexican Restaurant,Deli / Bodega,Thai Restaurant,Bakery,Sandwich Place,Spa,Donut Shop,Latin American Restaurant,Steakhouse,Café
11,Roosevelt Island,Park,Coffee Shop,Japanese Restaurant,Soccer Field,Gym / Fitness Center,Gym,School,Scenic Lookout,Sandwich Place,Farmers Market
20,Lower East Side,Chinese Restaurant,Latin American Restaurant,Café,Sandwich Place,Art Gallery,Bakery,Ramen Restaurant,Pizza Place,Japanese Restaurant,Diner
26,Morningside Heights,Bookstore,Park,American Restaurant,Coffee Shop,Sandwich Place,Burger Joint,Café,Deli / Bodega,Farmers Market,Outdoor Sculpture
30,Carnegie Hill,Coffee Shop,Café,Wine Shop,Pizza Place,Yoga Studio,Gym / Fitness Center,Bookstore,Cosmetics Shop,Gym,Bar


In [33]:
# cluster 4
manhattan_merged.loc[manhattan_merged['Cluster_labels'] == 3, 
                     manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Exhibit,Italian Restaurant,Gym / Fitness Center,Coffee Shop,Bakery,Yoga Studio,Spa,Hotel,Pizza Place,Juice Bar
10,Lenox Hill,Italian Restaurant,Coffee Shop,Cocktail Bar,Pizza Place,Gym,Gym / Fitness Center,Burger Joint,Sushi Restaurant,Café,Thai Restaurant
17,Chelsea,Bakery,Coffee Shop,American Restaurant,Art Gallery,Hotel,Seafood Restaurant,French Restaurant,Ice Cream Shop,Wine Shop,Cocktail Bar
18,Greenwich Village,Italian Restaurant,Clothing Store,Sushi Restaurant,Bakery,Cosmetics Shop,Boutique,Coffee Shop,Indian Restaurant,Sandwich Place,Gym
22,Little Italy,Café,Bakery,Chinese Restaurant,Italian Restaurant,Sandwich Place,Spa,Mediterranean Restaurant,Hotel,Optical Shop,Pizza Place
23,Soho,Clothing Store,Italian Restaurant,Boutique,Hotel,Shoe Store,Mediterranean Restaurant,Bakery,Art Gallery,Coffee Shop,Sporting Goods Shop
24,West Village,Italian Restaurant,New American Restaurant,Cocktail Bar,American Restaurant,Park,Cosmetics Shop,Coffee Shop,Jazz Club,Ice Cream Shop,Wine Bar
31,Noho,Cocktail Bar,Italian Restaurant,Pizza Place,French Restaurant,Rock Club,Sandwich Place,Bakery,Hotel,Grocery Store,Sushi Restaurant
38,Flatiron,Japanese Restaurant,American Restaurant,Italian Restaurant,New American Restaurant,Sporting Goods Shop,Spa,Mexican Restaurant,Mediterranean Restaurant,Wine Shop,Furniture / Home Store


In [34]:
# cluster_4
manhattan_merged.loc[manhattan_merged['Cluster_labels'] == 4, 
                     manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Bar,Park,Coffee Shop,Boat or Ferry,Gym / Fitness Center,Fountain,Harbor / Marina,Bistro,Heliport,Farmers Market
